In [1]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [2]:
client = OpenAI()

In [3]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [4]:
chat_completion.choices[0].message.content

'Hello! How are you today?'

In [12]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role":"system", "content": self.system})

    def __call__(self, message):
        self.messages.append({"role":"user", "content":message})
        result = self.execute()
        self.messages.append({"role":"assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.messages
        )
        print(completion.usage)
        return completion.choices[0].message.content

In [8]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [9]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [17]:
abot = Agent(prompt)

In [18]:
result = abot("How much does a toy poodle weigh?")
print(result)

CompletionUsage(completion_tokens=34, prompt_tokens=229, total_tokens=263)
Thought: I should find out the average weight of a Toy Poodle using average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


In [19]:
result = average_dog_weight("Toy Poodle")

In [20]:
result

'a toy poodles average weight is 7 lbs'

In [23]:
abot("Observation: {}".format(result))

CompletionUsage(completion_tokens=10, prompt_tokens=284, total_tokens=294)


'Answer: A Toy Poodle weighs 7 lbs'

In [25]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [26]:
abot = Agent(prompt)

In [27]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

CompletionUsage(completion_tokens=41, prompt_tokens=243, total_tokens=284)


'Thought: I can use the average_dog_weight action to find the average weight of each type of dog, and then add them together.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [28]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [30]:
abot(next_prompt)

CompletionUsage(completion_tokens=13, prompt_tokens=305, total_tokens=318)


'Action: average_dog_weight: Scottish Terrier\nPAUSE'

In [31]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [33]:
abot(next_prompt)

CompletionUsage(completion_tokens=27, prompt_tokens=381, total_tokens=408)


'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs (37 lbs + 20 lbs)'

In [35]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [45]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [48]:
query('what is the weight of a border collie and a scottish terrier combined?')

CompletionUsage(completion_tokens=35, prompt_tokens=238, total_tokens=273)
Thought: I should calculate the sum of the average weights of a Border Collie and a Scottish Terrier.

Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
CompletionUsage(completion_tokens=13, prompt_tokens=294, total_tokens=307)
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
CompletionUsage(completion_tokens=12, prompt_tokens=325, total_tokens=337)
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
CompletionUsage(completion_tokens=20, prompt_tokens=350, total_tokens=370)
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.
